In [1]:
pip install pyspark


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import sys
import csv
import time
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
from textblob import TextBlob
# Importing libraries
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
import pyspark.sql.types as tp
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from wordcloud import WordCloud
    
# initializing spark session
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
    

try:

##Function to obtain sentiment scores using Textblob 
   
	def senti(x):
		return TextBlob(x).sentiment.polarity 


    
##Obtaining the Sentiment Scores for all tweets       
	def analyze_sentiment(tweet):

		tweet_text = tweet.filter(lambda x: len(x) > 0)
		rowRdd = tweet_text.map(lambda w: Row(tweet=w))
		wordsDataFrame = spark.createDataFrame(rowRdd)
		wordpandasdf=wordsDataFrame.toPandas()
		wordpandasdf['senti_score'] = wordpandasdf['tweet'].apply(senti)
		score=wordpandasdf['senti_score']
##Writing the csv file to be read by Sentiment_score plot
		with open('data.csv', 'a') as f:
			score.to_csv(f, mode='a',header=False)
  

        
        ##Word Cloud part Starts Here    
		long_string = ','.join(list(wordpandasdf['tweet'].values))
		# Create a WordCloud object
		wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
		# Generate a word cloud    
		wordcloud.generate(long_string)
		# Visualize the word cloud
		wordcloud.to_image()
		wordcloud.to_file('Costco.png')

    

except : 
    print('No data')
    e = sys.exc_info()[0]
    print("This Error: %s" % e)
    
# initialize the streaming context 
ssc = StreamingContext(sc, batchDuration= 3)

# Create a DStream that will connect to hostname:port, like localhost:9099
lines = ssc.socketTextStream('localhost', 9099)

# split the tweet text by a keyword 'TWEET_APP' so that we can identify which set of words is from a single tweet
words = lines.flatMap(lambda line : line.split('TWEET_APP'))

##Filter to seperate tweets for a Particular Store (Costco or Walmart)
words=words.filter(lambda s:'costco' in s)
# get the predicted sentiments for the tweets received
words.foreachRDD(analyze_sentiment)

# Start the computation
ssc.start()             

# Wait for the computation to terminate
ssc.awaitTermination() 

['Niggas die and become a distant memory to most .. stay on course &amp; remain humble', '@kwesloh @TheCurrent I’m glad you got more than a feeling when you heard that old song playing', '@MysticxLipstick Amazing !!!!!🙏🙏🙏✨', 'somebody tag all ah mih ex them 😭', "The MMA media can't stop this event from happening but ESPN and Disney can. ", "We're in - in every sense of the clic… https://t.co/NlhYzxYNKg", 'I’m all for justice in their case.', '(Probably not the same trope of justice for the folks behind this website.)', 'Lost the one I’ve always searched for 💔', '@KJS407 Shits sucks dude. U still gonna go free will June 1st? I remember ur tweet lol', '@SenatorCollins Goodbye old lady. Remember Kavanaugh and the acquittal.', 'a friendly reminder that:', 'THATS NOT HOW SOCIAL DISTANCE WORKS https://t.co/XVT5ZlSq2p', '@avanasear That’s fair it is quite different but also I understand that being a thing for you', 'Send help @juanjsierrac', "@AGE2489 I can't stand Taco Bell. I prefer Del Tac

["All my friends keep telling me I just need to fuck someone new. Whenever I do, I'm gonna think of you.", 'Ninjas love to flirt when I already lost feelings for them. Like you’re too late baby', 'they do. you mf just be playin w us.', '@KateQFunny @DPSS_CalFresh @CAgovernor Can you get to NC....I got ur back.', '@webeunruly @Eslawal Did you see the hand twerk', '@david_barth @Acosta 😂😂😂 isn’t cute how you think you’re clever 🙄', '@msogotf Ok, thank you !', '@ndtv @sanket @sohitmishra99 @BDUTT @PrannoyRoyNDTV @ndtv @ravishndtv @sardesairajdeep Sir, Can you please raise th… https://t.co/Cfm8iJpp0P']
['@narendramodi @AmitShah In Mumbai pandemic is frighting. If Gujrat is calling armed forces then maybe here also to check social distancing', 'It would be such a privilege to see paul oakenfold. A big pioneer of trance.', 'Where in the hell is my Fiona Apple that likes Friday night football? Ladies...ladies...ladies.... https://t.co/ATqOnofxDh', 'We have to remember Trump has stacked the ju

['@gavinthomas hey nephew! https://t.co/10HtyCV1hp', '@kaylaxjefferson i don’t appreciate this energy you guys are putting into the air🙅🏻\u200d♀️🙅🏻\u200d♀️', 'Me after writing lyrics of a song in middle of every answer during exams😁', '#Engineering ', '#Engineering ', '#RGPV https://t.co/oq97HzBckQ', '@txradioguy @tedcruz @JoeNBC The people who use the term "TDS" or "Trump Derangement Syndrome" have zero self aware… https://t.co/gl8BamcW4D', 'Bhagyashree, 19, was killed by her husband Sahul Hameem, 31, in Mysuru. ', 'Bhagyashree had changed her name to Suhan… https://t.co/y5Ur371viq', 'I thought that we were fine', '@ModerateKarina Welcome', 'Idk if you can say this is real but his song is a lil bop', '@Big_Stu11 Yessir my nigga a down low Legend fasho..', 'Did I have 4 hrs worth of meetings today to talk about my thesis THAT I THOUGHT I WAS DONE WITH? Did I get comments… https://t.co/LCYPNIkJEc', '#skirt to amir right now..', '#carrt https://t.co/1qJomWLKCy', '#irunwithmaud #justicefo

['To know that you do not know is the best.', 'To pretend to know when you do not know is a disease. ', '@EmbryEthan I kind of laughed at the golf with no carts policy in IL. I don’t golf but I’ve gone enough to know you… https://t.co/CKXiVaVoPE', '@degsysinclair @tammyga12634437 @renecj @LeeConroy18 @GavinNewsom By filtered avatar, do you mean good photos? I ju… https://t.co/mHu8NWyRhv', 'Its in the silence that you hear your thoughts the loudest...of all the things you have either done wrong or right.', '@ImonDiamone See. Now you see why he hated ja rule and them. That beef will never die.', '@big_business_ Why don’t you want me to be happy ?', '@SankofaNYC @CreamoWildHair @bluemiiints You know what you’re talking about.']
['i need somebody to do a meal plan for me because this weight is catching up fast! 😭', 'This is satisfying', 'Love the mask! 👏🏽👏🏽👏🏽👏🏽👏🏽👏🏽👏🏽👏🏽#BruinFridays', "Who's down to have a genuine conversation?..  😣😌", 'look what is launching tomorrow, eeek so excited! this

['So explain to me yet again how these amazing programs who help some businesses and exclude a whole country full of… https://t.co/eX0xO0yvwV', 'He would’ve been 26 today #irunwithmaud @ Northridge, California https://t.co/YQvD3b9JeZ', 'Just took my Trek for a short ride I forgot how fun riding was and HAHA I😳 almost forgot to pop my cleats out coming to a stop', 'Late night post trying get some attention. https://t.co/b8XICgZFfr', '@Dsiree1 Aight', 'No doubt honest governance with empathy..', 'Nigga 6ix9ine broke the internet but Bobby Shmurda finna cure corona AND break the internet', '@TheVibeCloud @Chicken_Nater @SassyyR6 @shbzz @pokimanelol Not sure why you’re trying to insult me by calling me a… https://t.co/DZzxYFYace', 'Illusionist Roy Horn of famed Siegfried and Roy duo dies from coronavirus https://t.co/Nuc2rcHaD7', 'Why is the cat me @Kay_moonchild21', 'tipsy lol', '@arnellarmon For next Friday or Friday after next.  But we all no the original Friday was the best Friday Smok